In [1]:
!git clone https://github.com/OpenGVLab/InternVL.git

fatal: destination path 'InternVL' already exists and is not an empty directory.


In [2]:
# !huggingface-cli download --resume-download --local-dir-use-symlinks False OpenGVLab/InternVL2_5-1B-MPO --local-dir InternVL2_5-1B-MPO

# Dataset Creation

In [3]:
!pip install datasets

In [ ]:
# hugging face dataset download
from datasets import load_dataset
dataset = load_dataset("your_dataset", token="input_token", split='train[:100]')

In [5]:

dataset

Dataset({
    features: ['image', 'question', 'answer'],
    num_rows: 100
})

In [6]:
# dataset["image"]

In [7]:
import os
from PIL import Image
import requests
from io import BytesIO

# Step 1: Create dataset folder and image folder
dataset_folder = "dataset_folder"
image_folder = os.path.join(dataset_folder, "images")

os.makedirs(image_folder, exist_ok=True)

# Step 2: Save images to the 'images' folder
for idx, sample in enumerate(dataset):
    # Assuming dataset has an 'image' column with PIL Image or URL
    if isinstance(sample['image'], Image.Image):
        image = sample['image']
    else:
        # If it's a URL, download the image
        response = requests.get(sample['image']['url'])
        image = Image.open(BytesIO(response.content))

    # Save the image
    image_path = os.path.join(image_folder, f"image_{idx}.jpg")
    image.save(image_path)
    # print(f"Saved: {image_path}")

print("All images have been saved successfully!")


All images have been saved successfully!


In [8]:
final_dataset_json = []
relative_image_path = "/content/dataset_folder/images"
for idx, sample in enumerate(dataset):
    temp_json_dataset = {}

    x, y = dataset["image"][idx].size
    # Assuming dataset has an 'image' column with PIL Image or URL
    # print(sample["question"], sample["answer"])


    temp_json_dataset["id"] = idx
    temp_json_dataset["image"] = f"{relative_image_path}/image_{idx}.jpg"
    temp_json_dataset["width"] = x
    temp_json_dataset["height"] = y
    temp_json_dataset["conversations"] = [
      {
        "from": "human",
        "value": f"<image>\n{sample['question']}"
      },
      {
        "from": "gpt",
        "value": f"{sample['answer']}"
      }
    ]
    final_dataset_json.append(temp_json_dataset)

    # if idx == 2:
    #   break

    # break

In this format json file required
```
{
  "id": 0,
  "image": "images/00000000.jpg",
  "width": 897,
  "height": 1152,
  "conversations": [
    {
      "from": "human",
      "value": "<image>\nCan you extract any readable text from the image?"
    },
    {
      "from": "gpt",
      "value": "Dares Wins Vol. 5 Tommy's Heroes Vol. 6: For Tomorrow Vol. 7: Closing Time miniseries. Clark Kent is being interviewed about Superman's connection to notorious killer Tommy Monaghan. Taking the conversation..."
    }
  ]
}
```

In [9]:
# final_dataset_json

In [10]:
import json
with open('data.jsonl', 'w') as jsonl_file:
    for item in final_dataset_json:
        # Convert the dictionary to a JSON string
        json_line = json.dumps(item)

        # Write the JSON string followed by a newline
        jsonl_file.write(json_line + '\n')

In [11]:
# creating meta_data_file for data information
import json

meta_data = {
    "data": {
        "root": "/content/dataset_folder/images",
        "annotation": "/content/data.jsonl",
        "data_augment": False,
        "repeat_time": 1,
        "length": len(dataset)
    }
}
with open("/content/InternVL/internvl_chat/shell/data/train_metadata.jsonl", "w") as f:
    json.dump(meta_data, f)

In [12]:
import sys
import os

# Get the current script's directory or set a custom path
# current_path = os.path.abspath("/content/InternVL")

# # Add the directory to sys.path
# if current_path not in sys.path:
#     sys.path.append(current_path)

sys.path.append("/content/InternVL")

In [13]:
os.chdir('/content/InternVL/internvl_chat')

In [14]:
!pwd

/content/InternVL/internvl_chat


In [15]:
# !pip install flash-attn==2.0.2

In [16]:
!pip install -r /content/InternVL/requirements/internvl_chat.txt

  Using cached bitsandbytes-0.42.0-py3-none-any.whl.metadata (9.9 kB)
Using cached bitsandbytes-0.42.0-py3-none-any.whl (105.0 MB)
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.45.2
    Uninstalling bitsandbytes-0.45.2:
      Successfully uninstalled bitsandbytes-0.45.2


In [17]:
# !pip install deepspeed

In [18]:
!nvidia-smi

Fri Feb 21 15:39:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [19]:
# !GPUS=1 PER_DEVICE_BATCH_SIZE=1 sh /content/InternVL/internvl_chat/shell/internvl2.5_mpo/preference_optimization/internvl2_5_1b_qwen2_5_0_5b_dynamic_res_mpo.sh

In [20]:
# !pip install flash_attn exllamav2 -U

In [21]:
!pip install -U bitsandbytes

  Using cached bitsandbytes-0.45.2-py3-none-manylinux_2_24_x86_64.whl.metadata (5.8 kB)
Using cached bitsandbytes-0.45.2-py3-none-manylinux_2_24_x86_64.whl (69.7 MB)
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.42.0
    Uninstalling bitsandbytes-0.42.0:
      Successfully uninstalled bitsandbytes-0.42.0


In [22]:
# !pip install --upgrade transformers

In [23]:
!pip uninstall flash-attn
!pip install flash-attn --no-build-isolation --no-cache-dir

Found existing installation: flash_attn 2.7.4.post1
Uninstalling flash_attn-2.7.4.post1:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/flash_attn-2.7.4.post1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/flash_attn/*
    /usr/local/lib/python3.11/dist-packages/flash_attn_2_cuda.cpython-311-x86_64-linux-gnu.so
    /usr/local/lib/python3.11/dist-packages/hopper/*
Proceed (Y/n)? y
  Successfully uninstalled flash_attn-2.7.4.post1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 155.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.7.4.post1-cp311-cp311-linux_x86_64.whl size=187815463 sha256=d944fc7d2f962bce83fc4708c2fc0c21eaf8255962a0b350ae919362a51b7ef2
  Stored in directory: /tmp/pip-ephem-wheel-cache-o9gspuw3/wheels/3d/88/d8/284b89f56af7d5bf366b10d6b8e251ac8a7c7bf3f04203fb4f
Successfully built flash-attn


In [24]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoConfig
path = "OpenGVLab/InternVL2_5-1B"

In [25]:
config = AutoConfig.from_pretrained(path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for OpenGVLab/InternVL2_5-1B contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/OpenGVLab/InternVL2_5-1B.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:

config

In [27]:
config_dict = config.to_dict()

In [28]:
config_dict.keys()

dict_keys(['return_dict', 'output_hidden_states', 'output_attentions', 'torchscript', 'torch_dtype', 'use_bfloat16', 'tf_legacy_loss', 'pruned_heads', 'tie_word_embeddings', 'chunk_size_feed_forward', 'is_encoder_decoder', 'is_decoder', 'cross_attention_hidden_size', 'add_cross_attention', 'tie_encoder_decoder', 'max_length', 'min_length', 'do_sample', 'early_stopping', 'num_beams', 'num_beam_groups', 'diversity_penalty', 'temperature', 'top_k', 'top_p', 'typical_p', 'repetition_penalty', 'length_penalty', 'no_repeat_ngram_size', 'encoder_no_repeat_ngram_size', 'bad_words_ids', 'num_return_sequences', 'output_scores', 'return_dict_in_generate', 'forced_bos_token_id', 'forced_eos_token_id', 'remove_invalid_values', 'exponential_decay_length_penalty', 'suppress_tokens', 'begin_suppress_tokens', 'architectures', 'finetuning_task', 'id2label', 'label2id', 'tokenizer_class', 'prefix', 'bos_token_id', 'pad_token_id', 'eos_token_id', 'sep_token_id', 'decoder_start_token_id', 'task_specific_pa

In [29]:
config_dict["vision_config"]["use_flash_attn"] = False

In [ ]:
config_dict["llm_config"]

In [ ]:
# eager
config_dict

In [32]:
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    use_flash_attn=False,
    trust_remote_code=True).eval().cuda()

modeling_internvl_chat.py:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

modeling_intern_vit.py:   0%|          | 0.00/18.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL2_5-1B:
- modeling_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


conversation.py:   0%|          | 0.00/15.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL2_5-1B:
- conversation.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL2_5-1B:
- modeling_internvl_chat.py
- modeling_intern_vit.py
- conversation.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

In [33]:
import transformers
print(transformers.__version__)

4.37.2


In [34]:
file_path = "/content/InternVL/internvl_chat/shell/internvl2.5/2nd_finetune/internvl2_5_1b_dynamic_res_2nd_finetune_lora.sh"

# Define the strings to replace
old_string = "./shell/data/internvl_1_2_finetune_custom.json"
new_string = "/content/InternVL/internvl_chat/shell/data/train_metadata.jsonl"

# Read the file
with open(file_path, "r") as file:
    file_contents = file.read()

# Replace the string
file_contents = file_contents.replace(old_string, new_string)

# Write back to the same file
with open(file_path, "w") as file:
    file.write(file_contents)

print("Replacement done successfully!")


Replacement done successfully!


In [45]:
os.chdir('/')

In [62]:
os.chdir("/root/.cache/huggingface/hub/models--OpenGVLab--InternVL2_5-1B/snapshots/f27984381d99c1f2da11989d3216ca7b5bb51721/")

In [63]:
!ls -a

.			     configuration_internvl_chat.py  modeling_internvl_chat.py
..			     conversation.py		     model.safetensors
added_tokens.json	     generation_config.json	     special_tokens_map.json
config.json		     merges.txt			     tokenizer_config.json
configuration_intern_vit.py  modeling_intern_vit.py	     vocab.json


In [64]:
with open("config.json", "r") as file:
    config_data = json.load(file)


In [66]:
config_data["llm_config"]["_attn_implementation"] = "eager"

In [67]:
os.chdir('/content/InternVL/internvl_chat')
!pwd

/content/InternVL/internvl_chat


In [68]:
!GPUS=1 PER_DEVICE_BATCH_SIZE=1 sh /content/InternVL/internvl_chat/shell/internvl2.5/2nd_finetune/internvl2_5_1b_dynamic_res_2nd_finetune_lora.sh

+ GPUS=1
+ BATCH_SIZE=16
+ PER_DEVICE_BATCH_SIZE=1
+ GRADIENT_ACC=16
+ pwd
+ export PYTHONPATH=/env/python:/content/InternVL/internvl_chat
+ export MASTER_PORT=34229
+ export TF_CPP_MIN_LOG_LEVEL=3
+ export LAUNCHER=pytorch
+ OUTPUT_DIR=work_dirs/internvl_chat_v2_5/internvl2_5_1b_dynamic_res_2nd_finetune_lora
+ [ ! -d work_dirs/internvl_chat_v2_5/internvl2_5_1b_dynamic_res_2nd_finetune_lora ]
+ tee -a work_dirs/internvl_chat_v2_5/internvl2_5_1b_dynamic_res_2nd_finetune_lora/training_log.txt
+ torchrun --nnodes=1 --node_rank=0 --master_addr=127.0.0.1 --nproc_per_node=1 --master_port=34229 internvl/train/internvl_chat_finetune.py --model_name_or_path OpenGVLab/InternVL2_5-1B --conv_style internvl2_5 --use_fast_tokenizer False --output_dir work_dirs/internvl_chat_v2_5/internvl2_5_1b_dynamic_res_2nd_finetune_lora --meta_path /content/InternVL/internvl_chat/shell/data/train_metadata.jsonl --overwrite_output_dir True --force_image_size 448 --max_dynamic_patch 6 --down_sample_ratio 0.5 --drop

In [69]:
!pip freeze

absl-py==1.4.0
accelerate==0.34.2
aiohappyeyeballs==2.4.6
aiohttp==3.11.12
aiosignal==1.3.2
alabaster==1.0.0
albucore==0.0.23
albumentations==2.0.4
ale-py==0.10.1
altair==5.5.0
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.6.0
arviz==0.20.0
astropy==7.0.1
astropy-iers-data==0.2025.2.10.0.33.26
astunparse==1.6.3
atpublic==4.1.0
attrs==25.1.0
audioread==3.0.1
autograd==1.7.0
babel==2.17.0
backcall==0.2.0
beautifulsoup4==4.13.3
betterproto==2.0.0b6
bigframes==1.36.0
bigquery-magics==0.5.0
bitsandbytes==0.45.2
bleach==6.2.0
blinker==1.9.0
blis==0.7.11
blosc2==3.1.0
bokeh==3.6.3
Bottleneck==1.4.2
bqplot==0.12.44
branca==0.8.1
CacheControl==0.14.2
cachetools==5.5.1
catalogue==2.0.10
certifi==2025.1.31
cffi==1.17.1
chardet==5.2.0
charset-normalizer==3.4.1
chex==0.1.88
clarabel==0.10.0
click==8.1.8
cloudpathlib==0.20.0
cloudpickle==3.1.1
cmake==3.31.4
cmdstanpy==1.2.5
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confe